In [0]:
create table books_2
(book_id string, book_name string, author string, category string, cost double)
using csv
options(  
  delimiter = ';',
  path = 'dbfs:/mnt/demo-datasets/bookstore/books-csv/',
  header = 'true'
);

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore

In [0]:
drop table books_demo;

In [0]:
select * from books_2;

In [0]:
select book_id, count(*)
from books_2
group by book_id;

In [0]:
describe books_2;

In [0]:
create table books_demo as 
select * from books_2 
where book_id in ('B01','B02','B03','B04','B05','B06','B07','B08','B09');

In [0]:
SELECT * FROM books_demo;

In [0]:
%python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark-streaming") \
.getOrCreate()

In [0]:
%python
(spark.readStream.table("books_demo").createOrReplaceTempView("streaming_books_dat"))

In [0]:
drop view streaming_books_dat;

In [0]:
select * from streaming_books_dat;

In [0]:
create or replace Temp view streaming_authors_view as 
select author, count(book_id) as book_count
from streaming_books_dat
group by author;

In [0]:
select * from streaming_authors_view;

In [0]:
show tables;

In [0]:
%python
(spark.table("streaming_authors_view")
        .writeStream
        .trigger(processingTime="4 seconds")
        .outputMode("complete")
        .option("checkpointLocation","dbfs:/mnt/demo/streaming_authors_new")
        .table("authors_new"))

In [0]:
select * from authors_new;

In [0]:
INSERT INTO books_demo
values ("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
         ("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
         ("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35);

In [0]:
select * from authors_new;

In [0]:
INSERT INTO books_demo
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30),
("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35);

In [0]:
select * from authors_new;

In [0]:
%python
(spark.table("streaming_authors_view")
        .writeStream
        .trigger(availableNow=True)
        .outputMode("complete")
        .option("checkpointLocation","dbfs:/mnt/demo/streaming_authors_loc")
        .table("authors")
        .awaitTermination()
);

In [0]:
select * from authors_new;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-raw/

In [0]:
%python
(spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format","parquet")
        .option("cloudFiles.schemaLocation","dbfs:/mnt/demo/orders_checkpoint")
        .load("dbfs:/mnt/demo-datasets/bookstore/orders-raw")
      .writeStream
        .option("checkpointLocation","dbfs:/mnt/demo/orders_checkpoint")
        .table("orders_stream_dat")
        )

In [0]:
select * from orders_stream_dat;

In [0]:
select count(*) from orders_stream_dat;

In [0]:
%python

def get_index(dir):
    files = dbutils.fs.ls(dir)
    index = 0
    if files:
        file = max(files).name
        index = int(file.rsplit('.', maxsplit=1)[0])
    return index+1

    
def load_file(current_index):
    latest_file = f"{str(current_index).zfill(2)}.parquet"
    print(f"Loading {latest_file} file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_dir}/{latest_file}", f"{raw_dir}/{latest_file}")

    
def load_new_data(all=False):
    index = get_index(raw_dir)
    if index >= 10:
        print("No more data to load\n")

    elif all == True:
        while index <= 10:
            load_file(index)
            index += 1
    else:
        load_file(index)
        index += 1


streaming_dir = f"dbfs:/mnt/demo-datasets/bookstore/orders-streaming/"
raw_dir = f"dbfs:/mnt/demo-datasets/bookstore/orders-raw"
load_new_data(all=True)


In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-streaming/

In [0]:
select count(*) from orders_stream_dat;

In [0]:
describe history orders_stream_dat;

In [0]:
show tables;

In [0]:
drop table orders_stream_dat;

In [0]:
show tables;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-new/

In [0]:
show tables;

In [0]:
select * from orders;

In [0]:
describe table orders;

In [0]:
%python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark_Cluster").getOrCreate()

In [0]:
drop table new_orders;

In [0]:
%python
(spark.readStream
        .format('cloudFiles')
        .option('cloudFiles.format','parquet')
        .option('cloudFiles.schemaLocation','dbfs:/mnt/demo/schema-loc')
        .load('dbfs:/mnt/demo-datasets/bookstore/orders-')
      .writeStream
        .option('checkpointLocation','dbfs:/mnt/demo/schema-loc')
        .table("new_orders")
        );

In [0]:
select count(*) from new_orders;

In [0]:
%python

def get_index(dir):
    files = dbutils.fs.ls(dir)
    index = 0
    if files:
        file = max(files).name
        index = int(file.rsplit('.', maxsplit=1)[0])
    return index+1

    
def load_file(current_index):
    latest_file = f"{str(current_index).zfill(2)}.parquet"
    print(f"Loading {latest_file} file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_dir}/{latest_file}", f"{raw_dir}/{latest_file}")

    
def load_new_data(all=False):
    index = get_index(raw_dir)
    if index >= 10:
        print("No more data to load\n")

    elif all == True:
        while index <= 10:
            load_file(index)
            index += 1
    else:
        load_file(index)
        index += 1


streaming_dir = f"dbfs:/mnt/demo-datasets/bookstore/orders-new"
raw_dir = f"dbfs:/mnt/demo-datasets/bookstore/orders-streaming"
load_new_data(all=True)


In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-raw/

In [0]:
show tables;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/

In [0]:
%python
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/04.parquet")
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/05.parquet")
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/06.parquet")
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/07.parquet")
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/08.parquet")
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/09.parquet")
dbutils.fs.rm("dbfs:/mnt/demo-datasets/bookstore/orders-raw/10.parquet")

In [0]:
%python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark Structured Streaming").getOrCreate()

In [0]:
%python
(spark.readStream
            .format('cloudFiles')
            .option('cloudFiles.format','parquet')
            .option('cloudFiles.schemaLocation','dbfs:/mnt/demo/checkpoints')
            .load('dbfs:/mnt/demo-datasets/bookstore/orders-raw')
      .writeStream
            .option('checkpointLocation','dbfs:/mnt/demo/checkpoints')
            .table('orders_latest')
);

In [0]:
select count(*) from orders_latest;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-streaming

In [0]:
%python
dbutils.fs.cp('dbfs:/mnt/demo-datasets/bookstore/orders-streaming/04.parquet','dbfs:/mnt/demo-datasets/bookstore/orders-raw/04.parquet')

In [0]:
select count(*) from orders_latest;

In [0]:
%python
dbutils.fs.cp('dbfs:/mnt/demo-datasets/bookstore/orders-streaming/05.parquet','dbfs:/mnt/demo-datasets/bookstore/orders-raw/05.parquet')

In [0]:
select count(*) from orders_latest;

In [0]:
%fs ls dbfs:/mnt/demo/

In [0]:
%python
dbutils.fs.rm('dbfs:/mnt/demo/',recurse=True);

In [0]:
show tables;

In [0]:
%python
table_names = ["books_2", "books_demo", "csv_2", "csv_books", "csv_data","csv_data_3","csv_view_table","csv_vie_table_2","employee","temp_csv_table"]

for table_name in table_names:
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")

In [0]:
show tables;

In [0]:
%python
tables = ["authors","authors_new","csv_data__3","csv_view_table_2","transaction"]

for table in tables:
    spark.sql(f"Drop Table if exists {table}")

In [0]:
show tables;

In [0]:
select * from orders_latest;